Election de 2022 premier tour

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import missingno as msno
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

import os
os.listdir()

['election2017.ipynb',
 '.DS_Store',
 'enums',
 'utils',
 'models',
 'election2012.ipynb',
 'labo1.ipynb',
 '.gitignore',
 'scripts',
 'db',
 'labo2.ipynb',
 '.git',
 'main.py',
 'election2022.ipynb',
 'elections']

#### Importer la database

In [25]:
from db import SessionLocal
session = SessionLocal()

Import des datasets.

In [26]:
df = pd.read_excel('./elections/presidentielles-2017-1.xlsx')

Visualise toutes les colones

In [27]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

df.head(1000)

,Code du département,Libellé du département,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs,% Blancs/Ins,% Blancs/Vot,Nuls,% Nuls/Ins,% Nuls/Vot,Exprimés,% Exp/Ins,% Exp/Vot,Sexe,Nom,Prénom,Voix,% Voix/Ins,% Voix/Exp,Sexe.1,Nom.1,Prénom.1,Voix.1,% Voix/Ins.1,% Voix/Exp.1,Sexe.2,Nom.2,Prénom.2,Voix.2,% Voix/Ins.2,% Voix/Exp.2,Sexe.3,Nom.3,Prénom.3,Voix.3,% Voix/Ins.3,% Voix/Exp.3,Sexe.4,Nom.4,Prénom.4,Voix.4,% Voix/Ins.4,% Voix/Exp.4,Sexe.5,Nom.5,Prénom.5,Voix.5,% Voix/Ins.5,% Voix/Exp.5,Sexe.6,Nom.6,Prénom.6,Voix.6,% Voix/Ins.6,% Voix/Exp.6,Sexe.7,Nom.7,Prénom.7,Voix.7,% Voix/Ins.7,% Voix/Exp.7,Sexe.8,Nom.8,Prénom.8,Voix.8,% Voix/Ins.8,% Voix/Exp.8,Sexe.9,Nom.9,Prénom.9,Voix.9,% Voix/Ins.9,% Voix/Exp.9,Sexe.10,Nom.10,Prénom.10,Voix.10,% Voix/Ins.10,% Voix/Exp.10
0,1,Ain,415909,81530,19.60,334379,80.40,6342,1.52,1.90,2239,0.54,0.67,325798,78.33,97.43,F,LE PEN,Marine,81455,19.58,25.00,M,MACRON,Emmanuel,73692,17.72,22.62,M,FILLON,François,69804,16.78,21.43,M,MÉLENCHON,Jean-Luc,51736,12.44,15.88,M,DUPONT-AIGNAN,Nicolas,19788,4.76,6.07,M,HAMON,Benoît,16711,4.02,5.13,M,ASSELINEAU,François,3612,0.87,1.11,M,LASSALLE,Jean,3465,0.83,1.06,M,POUTOU,Philippe,3098,0.74,0.95,F,ARTHAUD,Nathalie,1842,0.44,0.57,M,CHEMINADE,Jacques,595,0.14,0.18
1,2,Aisne,375752,80183,21.34,295569,78.66,5047,1.34,1.71,2323,0.62,0.79,288199,76.70,97.51,F,LE PEN,Marine,102787,27.36,35.67,M,MACRON,Emmanuel,51693,13.76,17.94,M,MÉLENCHON,Jean-Luc,48959,13.03,16.99,M,FILLON,François,46985,12.50,16.30,M,DUPONT-AIGNAN,Nicolas,14652,3.90,5.08,M,HAMON,Benoît,12231,3.26,4.24,M,POUTOU,Philippe,3156,0.84,1.10,F,ARTHAUD,Nathalie,2764,0.74,0.96,M,LASSALLE,Jean,2265,0.60,0.79,M,ASSELINEAU,François,2171,0.58,0.75,M,CHEMINADE,Jacques,536,0.14,0.19
2,3,Allier,253946,54357,21.40,199589,78.60,4240,1.67,2.12,2556,1.01,1.28,192793,75.92,96.60,M,MACRON,Emmanuel,45744,18.01,23.73,F,LE PEN,Marine,43071,16.96,22.34,M,MÉLENCHON,Jean-Luc,38324,15.09,19.88,M,FILLON,François,36499,14.37,18.93,M,HAMON,Benoît,10639,4.19,5.52,M,DUPONT-AIGNAN,Nicolas,9819,3.87,5.09,M,LASSALLE,Jean,2988,1.18,1.55,M,POUTOU,Philippe,2328,0.92,1.21,F,ARTHAUD,Nathalie,1543,0.61,0.80,M,ASSELINEAU,François,1483,0.58,0.77,M,CHEMINADE,Jacques,355,0.14,0.18
3,4,Alpes-de-Haute-Provence,126578,24323,19.22,102255,80.78,1806,1.43,1.77,736,0.58,0.72,99713,78.78,97.51,F,LE PEN,Marine,24463,19.33,24.53,M,MÉLENCHON,Jean-Luc,22448,17.73,22.51,M,MACRON,Emmanuel,19960,15.77,20.02,M,FILLON,François,18442,14.57,18.50,M,HAMON,Benoît,4983,3.94,5.00,M,DUPONT-AIGNAN,Nicolas,4860,3.84,4.87,M,LASSALLE,Jean,1721,1.36,1.73,M,POUTOU,Philippe,1178,0.93,1.18,M,ASSELINEAU,François,932,0.74,0.93,F,ARTHAUD,Nathalie,521,0.41,0.52,M,CHEMINADE,Jacques,205,0.16,0.21
4,5,Hautes-Alpes,109907,20821,18.94,89086,81.06,1541,1.40,1.73,618,0.56,0.69,86927,79.09,97.58,M,MACRON,Emmanuel,18948,17.24,21.80,M,MÉLENCHON,Jean-Luc,18796,17.10,21.62,F,LE PEN,Marine,18474,16.81,21.25,M,FILLON,François,16645,15.14,19.15,M,HAMON,Benoît,5109,4.65,5.88,M,DUPONT-AIGNAN,Nicolas,4938,4.49,5.68,M,LASSALLE,Jean,1609,1.46,1.85,M,POUTOU,Philippe,1049,0.95,1.21,M,ASSELINEAU,François,783,0.71,0.90,F,ARTHAUD,Nathalie,411,0.37,0.47,M,CHEMINADE,Jacques,165,0.15,0.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,ZN,Nouvelle-Calédonie,189483,98257,51.86,91226,48.14,2969,1.57,3.25,1311,0.69,1.44,86946,45.89,95.31,M,FILLON,François,27065,14.28,31.13,F,LE PEN,Marine,25290,13.35,29.09,M,MACRON,Emmanuel,11089,5.85,12.75,M,HAMON,Benoît,8125,4.29,9.34,M,MÉLENCHON,Jean-Luc,7703,4.07,8.86,M,DUPONT-AIGNAN,Nicolas,2521,1.33,2.90,M,ASSELINEAU,François,2098,1.11,2.41,M,POUTOU,Philippe,1284,0.68,1.48,F,ARTHAUD,Nathalie,836,0.44,0.96,M,LASSALLE,Jean,695,0.37,0.80,M,CHEMINADE,Jacques,240,0.13,0.28
102,ZP,Polynés

Supprimer la colonne Etat saisie d’un DataFrame pandas.

In [28]:
df = df.drop(columns=["Etat saisie"], errors="ignore")

In [29]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

df.head(1000)

,Code du département,Libellé du département,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs,% Blancs/Ins,% Blancs/Vot,Nuls,% Nuls/Ins,% Nuls/Vot,Exprimés,% Exp/Ins,% Exp/Vot,Sexe,Nom,Prénom,Voix,% Voix/Ins,% Voix/Exp,Sexe.1,Nom.1,Prénom.1,Voix.1,% Voix/Ins.1,% Voix/Exp.1,Sexe.2,Nom.2,Prénom.2,Voix.2,% Voix/Ins.2,% Voix/Exp.2,Sexe.3,Nom.3,Prénom.3,Voix.3,% Voix/Ins.3,% Voix/Exp.3,Sexe.4,Nom.4,Prénom.4,Voix.4,% Voix/Ins.4,% Voix/Exp.4,Sexe.5,Nom.5,Prénom.5,Voix.5,% Voix/Ins.5,% Voix/Exp.5,Sexe.6,Nom.6,Prénom.6,Voix.6,% Voix/Ins.6,% Voix/Exp.6,Sexe.7,Nom.7,Prénom.7,Voix.7,% Voix/Ins.7,% Voix/Exp.7,Sexe.8,Nom.8,Prénom.8,Voix.8,% Voix/Ins.8,% Voix/Exp.8,Sexe.9,Nom.9,Prénom.9,Voix.9,% Voix/Ins.9,% Voix/Exp.9,Sexe.10,Nom.10,Prénom.10,Voix.10,% Voix/Ins.10,% Voix/Exp.10
0,1,Ain,415909,81530,19.60,334379,80.40,6342,1.52,1.90,2239,0.54,0.67,325798,78.33,97.43,F,LE PEN,Marine,81455,19.58,25.00,M,MACRON,Emmanuel,73692,17.72,22.62,M,FILLON,François,69804,16.78,21.43,M,MÉLENCHON,Jean-Luc,51736,12.44,15.88,M,DUPONT-AIGNAN,Nicolas,19788,4.76,6.07,M,HAMON,Benoît,16711,4.02,5.13,M,ASSELINEAU,François,3612,0.87,1.11,M,LASSALLE,Jean,3465,0.83,1.06,M,POUTOU,Philippe,3098,0.74,0.95,F,ARTHAUD,Nathalie,1842,0.44,0.57,M,CHEMINADE,Jacques,595,0.14,0.18
1,2,Aisne,375752,80183,21.34,295569,78.66,5047,1.34,1.71,2323,0.62,0.79,288199,76.70,97.51,F,LE PEN,Marine,102787,27.36,35.67,M,MACRON,Emmanuel,51693,13.76,17.94,M,MÉLENCHON,Jean-Luc,48959,13.03,16.99,M,FILLON,François,46985,12.50,16.30,M,DUPONT-AIGNAN,Nicolas,14652,3.90,5.08,M,HAMON,Benoît,12231,3.26,4.24,M,POUTOU,Philippe,3156,0.84,1.10,F,ARTHAUD,Nathalie,2764,0.74,0.96,M,LASSALLE,Jean,2265,0.60,0.79,M,ASSELINEAU,François,2171,0.58,0.75,M,CHEMINADE,Jacques,536,0.14,0.19
2,3,Allier,253946,54357,21.40,199589,78.60,4240,1.67,2.12,2556,1.01,1.28,192793,75.92,96.60,M,MACRON,Emmanuel,45744,18.01,23.73,F,LE PEN,Marine,43071,16.96,22.34,M,MÉLENCHON,Jean-Luc,38324,15.09,19.88,M,FILLON,François,36499,14.37,18.93,M,HAMON,Benoît,10639,4.19,5.52,M,DUPONT-AIGNAN,Nicolas,9819,3.87,5.09,M,LASSALLE,Jean,2988,1.18,1.55,M,POUTOU,Philippe,2328,0.92,1.21,F,ARTHAUD,Nathalie,1543,0.61,0.80,M,ASSELINEAU,François,1483,0.58,0.77,M,CHEMINADE,Jacques,355,0.14,0.18
3,4,Alpes-de-Haute-Provence,126578,24323,19.22,102255,80.78,1806,1.43,1.77,736,0.58,0.72,99713,78.78,97.51,F,LE PEN,Marine,24463,19.33,24.53,M,MÉLENCHON,Jean-Luc,22448,17.73,22.51,M,MACRON,Emmanuel,19960,15.77,20.02,M,FILLON,François,18442,14.57,18.50,M,HAMON,Benoît,4983,3.94,5.00,M,DUPONT-AIGNAN,Nicolas,4860,3.84,4.87,M,LASSALLE,Jean,1721,1.36,1.73,M,POUTOU,Philippe,1178,0.93,1.18,M,ASSELINEAU,François,932,0.74,0.93,F,ARTHAUD,Nathalie,521,0.41,0.52,M,CHEMINADE,Jacques,205,0.16,0.21
4,5,Hautes-Alpes,109907,20821,18.94,89086,81.06,1541,1.40,1.73,618,0.56,0.69,86927,79.09,97.58,M,MACRON,Emmanuel,18948,17.24,21.80,M,MÉLENCHON,Jean-Luc,18796,17.10,21.62,F,LE PEN,Marine,18474,16.81,21.25,M,FILLON,François,16645,15.14,19.15,M,HAMON,Benoît,5109,4.65,5.88,M,DUPONT-AIGNAN,Nicolas,4938,4.49,5.68,M,LASSALLE,Jean,1609,1.46,1.85,M,POUTOU,Philippe,1049,0.95,1.21,M,ASSELINEAU,François,783,0.71,0.90,F,ARTHAUD,Nathalie,411,0.37,0.47,M,CHEMINADE,Jacques,165,0.15,0.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,ZN,Nouvelle-Calédonie,189483,98257,51.86,91226,48.14,2969,1.57,3.25,1311,0.69,1.44,86946,45.89,95.31,M,FILLON,François,27065,14.28,31.13,F,LE PEN,Marine,25290,13.35,29.09,M,MACRON,Emmanuel,11089,5.85,12.75,M,HAMON,Benoît,8125,4.29,9.34,M,MÉLENCHON,Jean-Luc,7703,4.07,8.86,M,DUPONT-AIGNAN,Nicolas,2521,1.33,2.90,M,ASSELINEAU,François,2098,1.11,2.41,M,POUTOU,Philippe,1284,0.68,1.48,F,ARTHAUD,Nathalie,836,0.44,0.96,M,LASSALLE,Jean,695,0.37,0.80,M,CHEMINADE,Jacques,240,0.13,0.28
102,ZP,Polynés

In [30]:
df.shape

(106, 82)

Recuperer les information généraux sur les élections départemental

In [31]:
df_infos_general = df.iloc[:, :16]
df_infos_general.head()

,Code du département,Libellé du département,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs,% Blancs/Ins,% Blancs/Vot,Nuls,% Nuls/Ins,% Nuls/Vot,Exprimés,% Exp/Ins,% Exp/Vot
0,1,Ain,415909,81530,19.60,334379,80.40,6342,1.52,1.90,2239,0.54,0.67,325798,78.33,97.43
1,2,Aisne,375752,80183,21.34,295569,78.66,5047,1.34,1.71,2323,0.62,0.79,288199,76.70,97.51
2,3,Allier,253946,54357,21.40,199589,78.60,4240,1.67,2.12,2556,1.01,1.28,192793,75.92,96.60
3,4,Alpes-de-Haute-Provence,126578,24323,19.22,102255,80.78,1806,1.43,1.77,736,0.58,0.72,99713,78.78,97.51
4,5,Hautes-Alpes,109907,20821,18.94,89086,81.06,1541,1.40,1.73,618,0.56,0.69,86927,79.09,97.58


Renommer les colonnes des information généraux sur les élections départemental

In [32]:
df_infos_general.columns = [
    "code_dept", "nom_dept", "nb_inscrits", "nb_abstentions",
    "pct_abstentions", "nb_votants", "pct_votants", "nb_blancs",
    "pct_blancs_inscrits", "pct_blancs_votants", "nb_nuls",
    "pct_nuls_inscrits", "pct_nuls_votants", "nb_exprimes",
    "pct_exprimes_inscrits", "pct_exprimes_votants"
]
df_infos_general.head()

,code_dept,nom_dept,nb_inscrits,nb_abstentions,pct_abstentions,nb_votants,pct_votants,nb_blancs,pct_blancs_inscrits,pct_blancs_votants,nb_nuls,pct_nuls_inscrits,pct_nuls_votants,nb_exprimes,pct_exprimes_inscrits,pct_exprimes_votants
0,1,Ain,415909,81530,19.60,334379,80.40,6342,1.52,1.90,2239,0.54,0.67,325798,78.33,97.43
1,2,Aisne,375752,80183,21.34,295569,78.66,5047,1.34,1.71,2323,0.62,0.79,288199,76.70,97.51
2,3,Allier,253946,54357,21.40,199589,78.60,4240,1.67,2.12,2556,1.01,1.28,192793,75.92,96.60
3,4,Alpes-de-Haute-Provence,126578,24323,19.22,102255,80.78,1806,1.43,1.77,736,0.58,0.72,99713,78.78,97.51
4,5,Hautes-Alpes,109907,20821,18.94,89086,81.06,1541,1.40,1.73,618,0.56,0.69,86927,79.09,97.58


In [33]:
df_infos_general.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   code_dept              106 non-null    object 
 1   nom_dept               106 non-null    object 
 2   nb_inscrits            106 non-null    int64  
 3   nb_abstentions         106 non-null    int64  
 4   pct_abstentions        106 non-null    float64
 5   nb_votants             106 non-null    int64  
 6   pct_votants            106 non-null    float64
 7   nb_blancs              106 non-null    int64  
 8   pct_blancs_inscrits    106 non-null    float64
 9   pct_blancs_votants     106 non-null    float64
 10  nb_nuls                106 non-null    int64  
 11  pct_nuls_inscrits      106 non-null    float64
 12  pct_nuls_votants       106 non-null    float64
 13  nb_exprimes            106 non-null    int64  
 14  pct_exprimes_inscrits  106 non-null    float64
 15  pct_ex

Suppression des données redondantes, risque de multicolinéarité en ML

In [34]:
colonnes_base = [
    "code_dept", "nom_dept",
    "nb_inscrits", "nb_abstentions", "nb_votants", 
    "nb_blancs", "nb_nuls"
]

df_infos_general_base = df_infos_general[colonnes_base]
df_infos_general_base.head()

,code_dept,nom_dept,nb_inscrits,nb_abstentions,nb_votants,nb_blancs,nb_nuls
0,1,Ain,415909,81530,334379,6342,2239
1,2,Aisne,375752,80183,295569,5047,2323
2,3,Allier,253946,54357,199589,4240,2556
3,4,Alpes-de-Haute-Provence,126578,24323,102255,1806,736
4,5,Hautes-Alpes,109907,20821,89086,1541,618


In [35]:
df_infos_general_base.shape

(106, 7)

Si les colonnes nb_blancs et nb_nuls existent, les fusionées.

In [36]:

if {"nb_blancs", "nb_nuls"}.issubset(df_infos_general_base.columns):
    df_infos_general_base["nb_blancs_nuls"] = df_infos_general_base["nb_blancs"].fillna(0) + df_infos_general_base["nb_nuls"].fillna(0)
    df_infos_general_base = df_infos_general_base.drop(columns=["nb_blancs", "nb_nuls"])

df_infos_general_base.head()

/tmp/ipykernel_38765/3627291289.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_infos_general_base["nb_blancs_nuls"] = df_infos_general_base["nb_blancs"].fillna(0) + df_infos_general_base["nb_nuls"].fillna(0)


,code_dept,nom_dept,nb_inscrits,nb_abstentions,nb_votants,nb_blancs_nuls
0,1,Ain,415909,81530,334379,8581
1,2,Aisne,375752,80183,295569,7370
2,3,Allier,253946,54357,199589,6796
3,4,Alpes-de-Haute-Provence,126578,24323,102255,2542
4,5,Hautes-Alpes,109907,20821,89086,2159


In [37]:
df_infos_general_base.shape

(106, 6)

In [38]:
df_departement = df_infos_general_base.iloc[:, :2]
df_departement.head()

,code_dept,nom_dept
0,1,Ain
1,2,Aisne
2,3,Allier
3,4,Alpes-de-Haute-Provence
4,5,Hautes-Alpes


In [39]:
df_stat_elections = df_infos_general_base.drop(columns=["nom_dept"])
df_stat_elections.head()

,code_dept,nb_inscrits,nb_abstentions,nb_votants,nb_blancs_nuls
0,1,415909,81530,334379,8581
1,2,375752,80183,295569,7370
2,3,253946,54357,199589,6796
3,4,126578,24323,102255,2542
4,5,109907,20821,89086,2159


Tratification pour consever les proportions

In [40]:
import pandas as pd

# Colonnes globales que l'on veut garder
cols_globales = ['Code du département', 'Libellé du département']

# Colonnes répétitives candidats
df_candidates = df.iloc[:, 16:]  # après % Exp/Vot
cols_per_candidate = 6
num_candidates = df_candidates.shape[1] // cols_per_candidate

dfs_list = []

for i in range(num_candidates):
    start_col = i * cols_per_candidate
    end_col = start_col + cols_per_candidate
    df_cand = df_candidates.iloc[:, start_col:end_col].copy()
    df_cand.columns = ['Sexe', 'Nom', 'Prenom', 'Voix', '% Voix/Ins', '% Voix/Exp']
    
    # Ajouter les informations de département
    df_cand = pd.concat([df[cols_globales], df_cand], axis=1)
    
    dfs_list.append(df_cand)

# Combiner tous les candidats
df_simple = pd.concat(dfs_list, ignore_index=True)

# Colonnes à garder
cols_to_keep = ['Code du département', 'Libellé du département', 'Sexe', 'Nom', 'Prenom', 'Voix']

# Nouveau dataframe simplifié
df_candidat_resultat = df_simple[cols_to_keep]

# Afficher toutes les lignes
pd.set_option('display.max_rows', 10)  # ou un nombre supérieur à 100
pd.set_option('display.max_columns', None)  # pour voir toutes les colonnes
pd.set_option('display.width', None)        # pour éviter la coupure à la largeur de l'écran


# Affichage des 5 premières lignes
df_candidat_resultat.head(1000)


,Code du département,Libellé du département,Sexe,Nom,Prenom,Voix
0,1,Ain,F,LE PEN,Marine,81455
1,2,Aisne,F,LE PEN,Marine,102787
2,3,Allier,M,MACRON,Emmanuel,45744
3,4,Alpes-de-Haute-Provence,F,LE PEN,Marine,24463
4,5,Hautes-Alpes,M,MACRON,Emmanuel,18948
...,...,...,...,...,...,...
995,41,Loir-et-Cher,M,ASSELINEAU,François,1516
996,42,Loire,F,ARTHAUD,Nathalie,2547
997,43,Haute-Loire,F,ARTHAUD,Nathalie,1043
998,44,Loire-Atlantique,F,ARTHAUD,Nathalie,4785


In [ ]:
from datetime import date

from utils.election_importer import ElectionImporter
from enums.type_election import TypeElection

# =====================================================
# Instanciation
# =====================================================
importer = ElectionImporter()

# =====================================================
# Élection
# =====================================================
#election = importer.get_or_create_election(
#    election_date=date(2017, 4, 23),
#    type_election=TypeElection.PRESIDENTIELLE,
#    tour=1
#)

election = importer.get_or_create_election(
    election_date=date(2017, 5, 7),
    type_election=TypeElection.PRESIDENTIELLE,
    tour=2
)

# =====================================================
# Départements
# =====================================================
#df_departement = edf.df_departement
if df_departement is not None and not df_departement.empty:
    importer.import_departements(df_departement)

# =====================================================
# Statistiques par département
# =====================================================
#df_stat_elections = edf.df_stat_elections
if df_stat_elections is not None and not df_stat_elections.empty:
    importer.import_stats(df_stat_elections, election.id)

# =====================================================
# Candidats & résultats
# =====================================================
#df_candidat_resultat = edf.df_candidat_resultat
if df_candidat_resultat is not None and not df_candidat_resultat.empty:
    importer.import_candidats_resultats(df_candidat_resultat, election.id)

print("✅ Import élection 2017 – Tour 1 terminé avec succès")


2025-12-26 11:54:29,831 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-26 11:54:29,834 INFO sqlalchemy.engine.Engine SELECT elections.id AS elections_id, elections.date AS elections_date, elections.type_election AS elections_type_election, elections.tour AS elections_tour 
FROM elections 
WHERE elections.date = %(date_1)s AND elections.type_election = %(type_election_1)s AND elections.tour = %(tour_1)s 
 LIMIT %(param_1)s
2025-12-26 11:54:29,836 INFO sqlalchemy.engine.Engine [cached since 234.8s ago] {'date_1': datetime.date(2017, 4, 23), 'type_election_1': 'PRESIDENTIELLE', 'tour_1': 1, 'param_1': 1}
2025-12-26 11:54:29,847 INFO sqlalchemy.engine.Engine SELECT departements.code_dept AS departements_code_dept, departements.nom_dept AS departements_nom_dept 
FROM departements 
WHERE departements.code_dept = %(pk_1)s
2025-12-26 11:54:29,847 INFO sqlalchemy.engine.Engine [cached since 234.7s ago] {'pk_1': '01'}
2025-12-26 11:54:29,854 INFO sqlalchemy.engine.Engine SELECT departeme